In [23]:
import numpy as np
import pandas as pd
from termcolor import colored

The question we want to answer here is the following:

Is this data ready for ETL?

We have 2 files:
1. subjects
2. biomarkers (tvs)

First lets start with the subjects file

We are told there are a total of 511 subjects and 4 trial arms. Furthermore, we are also told the trial arms are 
1. P (placebo)
2. 1.3
3. 1.6
4. 1.9 

Heres how I am checking the subjects file:
1. Listsing all the columns and making sure that the required columns exist 
2. Required columns dont contain any null values
3. Unique columns only contain unique values
4. Making sure there are 511 subjects
5. Making sure there are only 4 trial arm ids
6. Making sure that the trial arms are the 4 mentioned above

Now the biomarkers or tvs file

There are still 511 subjects. We are told there are 7 measurement values:
1. sld
2. dsld
3. target_pd 
4. non_target_pd
5. new_lesion_pd
6. rebound_pd
7. any_pd

The first 2 are numerical, and the rest are boolean

Heres how I am checking the tvs file:
1. Listsing all the columns and making sure that the required columns exist 
2. Required columns dont contain any null values
3. Unique columns only contain unique values
4. Making sure there are 511 subjects
5. Making sure all 7 measurements exist
6. Making sure the values for each measurement type matches the correct format


Additional quesions:
1. Do all subjects need to have all 7 measurements
2. Do we need to have equal number of entries for each measurement
3. Do we need to have equal number of entries for each subject
4. Do we need to have equal number of measurements for each subject (subjec a had 70 entires, but subject b has 7. is that ok
5. Do we need to have equal number of each measurement for each subject (subject A has 5 entires for sld, but only 4 for dsld. is that ok)
6. acceptable ranges for numerical columns
7. ranges for numerical format

In [98]:
required = {
    'subjects': {
        'subjects': 511,
        'required': ['id', 'individual_id', 'trial_arm_id'],
        'unique': ['id'],
        'trials_arms': ['P', '1.3', '1.6', '1.9']
    },
    'tvs': {
        'subjects': 511,
        'trial_arms': 4,
        'required': ['id', 'subject_id', 'trial_day', 'measurement_name', 'measurement_value'],
        'unique': ['id'],
        'measurements': {
            'name': ['sld', 'dsld', 'target_pd', 'non_target_pd', 'new_lesion_pd', 'rebound_pd', 'any_pd'],
            'values': {
                'sld': float,
                'dsld': float,
                'target_pd': bool,
                'non_target_pd': bool,
                'new_lesion_pd': bool,
                'rebound_pd': bool,
                'any_pd': bool
            },
            'ranges': {
                'sld': [],
                'dsld': [],
            }
        }
    }
}

### TVS (Biomarkers)

In [99]:
df_tv = pd.read_csv('tvs.csv')
df_tv.head()

,Unnamed: 0,id,created_at,subject_id,trial_day,measurement_name,measurement_unit,measurement_value,comment,params,time
0,0,06e60ca4-43b8-4003-a6f3-b45baf056b69,2022-11-09T15:32:13.501405,2a3375d2-2faf-4016-be37-fe7a24f9f2cd,0.0,sld,mm,303.673363,NaN,{'visit_desc': 'Visit 1'},NaN
1,1,6f658700-d518-4c8a-957f-d0861df3032f,2022-11-09T15:32:13.501405,2a3375d2-2faf-4016-be37-fe7a24f9f2cd,77.0,sld,mm,152.275501,NaN,{'visit_desc': 'Visit 2'},NaN
2,2,bc379aa1-d8b1-4844-bc88-b26ab09732a3,2022-11-09T15:32:13.501405,2a3375d2-2faf-4016-be37-fe7a24f9f2cd,161.0,sld,mm,117.237736,NaN,{'visit_desc': 'Visit 3'},NaN
3,3,0ede290c-cb0b-459e-a520-682783e5ab82,2022-11-09T15:32:13.501405,2a3375d2-2faf-4016-be37-fe7a24f9f2cd,245.0,sld,mm,102.165337,NaN,{'visit_desc': 'Visit 4'},NaN
4,4,0ceb6525-ab5d-413c-bae6-53a74342e8aa,2022-11-09T15:32:13.501405,2a3375d2-2faf-4016-be37-fe7a24f9f2cd,336.0,sld,mm,93.563112,NaN,{'visit_desc': 'Visit 5'},NaN


In [128]:
def required_columns(df, file):
    columns = set(df.columns)
    for column in required[file]['required']:
        if (column not in columns):
            print('{} is required, but NOT found. Please fix'.format(column))
            return
    print('All required columns exist')

def required_columns_na(df, file):
    columns_na_dict = df.isnull().sum().to_dict()
    for column in required[file]['required']:
        if (columns_na_dict[column] > 0):
            print('{} has NULL values. Please fix'.format(column))
            return
    print('Required columns contain no NULL values')
    
def unique_columns(df, file):
    for column in required[file]['unique']:
        if not (df[column].is_unique):
            print('{} contains duplicates. This is NOT allowed. Please fix'.format(column))
            return
    print('No duplicates found in unique columns')
    
def num_subjects(df, file):
    subjects = df['subject_id'].unique()
    if (len(subjects) != required[file]['subjects'] * required[file]['trial_arms']):
        print('Number of subjects NOT the same as defined. Please fix')
        return
    print('Correct number of subjects')
    
def num_measurements(df, file):
    measurements = set(df['measurement_name'].unique())
    for measurement in required[file]['measurements']['name']:
        if (measurement not in measurements):
            print('{} NOT in measurements. Please fix'. format(measurement))
            return
    print('All measurements found')
    
def measurement_format(df, file):
    for measurement, measurement_type in required[file]['measurements']['values'].items():
        df_dtype = df['measurement_value'][df['measurement_name'] == measurement].dtype
        if (measurement_type == bool):
            values = set(df['measurement_value'][df['measurement_name'] == measurement].unique())
            if (sum(values) == 0.0 or sum(values) == 1.0):
                df_dtype = bool
        if (df_dtype != measurement_type):
            print('{} has the wrong format. It is {}, but should be {}'.format(
                measurement, df_dtype, measurement_type
            ))
            return
    print('All measurement values are good')
    

In [129]:
required_columns(df_tv, 'tvs')
required_columns_na(df_tv, 'tvs')
unique_columns(df_tv, 'tvs')
num_subjects(df_tv, 'tvs')
num_measurements(df_tv, 'tvs')
measurement_format(df_tv, 'tvs')

All required columns exist
Required columns contain no NULL values
No duplicates found in unique columns
Correct number of subjects
All measurements found
All measurement values are good


### Subjects

In [6]:
df_s = pd.read_csv('subjects.csv')
df_s.head()

,Unnamed: 0,id,individual_id,created_at,country,age_min,age_max,sex,race,ethnicity,smoking_exposure,performance,first_treatment_day,baseline_weight_min,baseline_weight_max,last_evaluable_assessment,trial_arm_id,params
0,0,c3b37cc7-2997-495b-9604-62218ab99f0f,1_1.3,2022-11-09T15:32:13.501405,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,542586b6-41ba-40cd-9a5b-1806ea448dfa,NaN
1,1,9da60426-d5f6-444e-8156-1bee89eb3564,1_1.6,2022-11-09T15:32:13.501405,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dcdbea03-d995-4565-9929-6ea9bdcecc0c,NaN
2,2,7717f49d-45d4-4975-a751-905bd0d080c9,1_1.9,2022-11-09T15:32:13.501405,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1fb13152-be75-4523-a93e-feaa9612d05a,NaN
3,3,b3b920d4-375f-401a-bcac-9c9955d4da58,1_P,2022-11-09T15:32:13.501405,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,878d0414-42c0-4a7e-ae22-427613aad41f,NaN
4,4,6c3d642f-b97b-4f0d-b4c3-2f0d0b271133,2_1.3,2022-11-09T15:32:13.501405,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,542586b6-41ba-40cd-9a5b-1806ea448dfa,NaN


In [65]:
def required_columns(df, file):
    columns = set(df.columns)
    for column in required[file]['required']:
        if (column not in columns):
            print('{} is required, but NOT found. Please fix'.format(column))
            return
    print('All required columns exist')

def required_columns_na(df, file):
    columns_na_dict = df.isnull().sum().to_dict()
    for column in required[file]['required']:
        if (columns_na_dict[column] > 0):
            print('{} has NULL values. Please fix'.format(column))
            return
    print('Required columns contain no NULL values')
    
def unique_columns(df, file):
    for column in required[file]['unique']:
        if not (df[column].is_unique):
            print('{} contains duplicates. This is NOT allowed. Please fix'.format(column))
            return
    print('No duplicates found in unique columns')
    
def num_subjects(df, file):
    subjects = set(df['individual_id'].apply(lambda x: x.split('_')[0]))
    if (len(subjects) != required[file]['subjects']):
        print('Number of subjects NOT the same as defined. Please fix')
        return
    print('Correct number of subjects')

def trial_arms(df, file):
    trial_arms_ids = set(df['trial_arm_id'].unique())
    if (len(trial_arms_ids) != len(required[file]['trials_arms'])):
        print('Number of trial arms NOT the same as defined. Please fix')
        return
    print('Correct number of trial arms')
    
    trial_arms = set(df['individual_id'].apply(lambda x: x.split('_')[1]))
    for trial_arm in required[file]['trials_arms']:
        if trial_arm not in trial_arms:
            print('{} trial arm NOT found. Please fix'.format(trial_arm))
            return
    print('All correct trial arms found')

In [66]:
required_columns(df_s, 'subjects')
required_columns_na(df_s, 'subjects')
unique_columns(df_s, 'subjects')
num_subjects(df_s, 'subjects')
trial_arms(df_s, 'subjects')

All required columns exist
Required columns contain no NULL values
No duplicates found in unique columns
Correct number of subjects
Correct number of trial arms
All correct trial arms found
